
<a id='chap-tpacp'></a>

# Exemples pratiques - Principal Component Analysis for Dimensionality Reduction in Python

**L’objectif** de cette exemlpe pratique est de présenter l’utilisation des fonctionnalités de Scikit-learn concernant l’ACP, ainsi que de contribuer à une meilleure compréhension de cette méthode d’analyse de données dans le processuse pré-traitement. 
Après une brève présentation de la classe PCA de Scikit-learn, nous examinons d’abord des données générées de façon contrôlée afin de mieux comprendre l’impact de la distribution des données sur les résultats de l’ACP. Deux exemples, basés sur des données réelles, sont ensuite repris et examinés.

## L’ACP dans Scikit-learn

Dans Scikit-learn, l’analyse en composantes principales (ACP) est mise en œuvre dans la classe `sklearn.decomposition.PCA` (voir [http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)). 

Pour définir l’analyse on appelle `PCA(n_components=None, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto', random_state=None)`. 
Les paramètres d’appel sont :

> - `n_components` : nombre de composantes à conserver (par défaut toutes, c’est à dire `min(n_samples, n_features)` où `n_samples` est le nombre de lignes de la matrice de données et `n_features` le nombre de colonnes).  
- `copy` : si `False`, la matrice de données est écrasée par les données transformées (par défaut `True`).  
- `whiten` : si `True`, les données transformées sont modifiées pour que les projections sur les axes principaux présentent une variance unitaire (par défaut `False`).  
- `svd_solver` : solveur à employer parmi `'auto'`, `'full'`, `'arpack'`, `'randomized'` (par défaut `'auto'`).  
- `tol`, `iterated_power`, `random_state` : paramètres optionnels pour solveurs spécifiques.  



Les attributs accessibles sont les suivants :

> - `explained_variance_` : array `[n_components]` dans lequel on trouve les valeurs propres en ordre décroissant.  
- `components_` : array `[n_components, n_features]` dans lequel chaque ligne correspond à un vecteur propre ; l’ordre est celui des valeurs propres.  
- `explained_variance_ratio_` : array `[n_components]` avec les valeurs propres exprimées en pourcentage de la variance expliquée.  
- `mean_` : array `[n_features]` contenant les moyennes des variables calculées lors de l’ACP.  
- `n_components_` : nombre estimé de composantes.  
- `noise_variance_` : variance du bruit estimée suivant l’approche de l’ACP probabiliste (voir lien ci-dessus).  



Les méthodes qui peuvent être employées :

> - `fit(X[, y])` : trouver les axes principaux et les valeurs propres associés à partir de la matrice de données X.  
- `fit_transform(X[, y])` : trouver les axes principaux et les valeurs propres associés, appliquer la projection sur les axes principaux (avec stockage des résultats dans la matrice de données X si `copy=False` et dans une autre matrice si `copy=True`).  
- `get_covariance()` : calculer les covariances à partir de la matrice de données.  
- `get_params([deep])` : retourner les paramètres de l’estimateur.  
- `get_precision()` : calculer la matrice de précision des données.  
- `inverse_transform(X[, y])` : re-projeter les données transformées dans l’espace initial.  
- `score(X[, y])` : valeur moyenne de la log-vraisemblance pour toutes les observations.  
- `score_samples(X)` : log-vraisemblance pour chaque observation.  
- `set_params(**params)` : donner des valeurs aux paramètres de l’estimateur.  
- `transform(X[, y])` : appliquer la projection sur les axes principaux (avec stockage des résultats dans la matrice de données X si `copy=False` et dans une autre matrice si `copy=True`).  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## ACP des données sur le sommeil des mammifères

L’ACP sera d’abord appliquée aux données concernant le sommeil des mammifères (ci joint le fichier csv mammals.csv).

In [2]:
data=pd.read_csv("mammals.csv", delimiter=';', usecols=[1,2,3,4,5,6,7,8,9,10])#, skiprows=1

In [3]:
data

,BodyW,BrainW,SWS,PS,TS,LifeSpan,GT,PI,SEI,ODI
0,6654.000,5712.0,8.35,1.8,3.30,38.6,645.0,3,5,3
1,1.000,6.6,6.30,2.0,8.30,4.5,42.0,3,1,3
2,3.385,44.5,8.35,1.8,12.50,14.0,60.0,1,1,1
3,0.920,5.7,8.35,1.8,16.50,15.1,25.0,5,2,3
4,2547.000,4603.0,2.10,1.8,3.90,69.0,624.0,3,5,4
...,...,...,...,...,...,...,...,...,...,...
57,2.000,12.3,4.90,0.5,5.40,7.5,200.0,3,1,3
58,0.104,2.5,13.20,2.6,15.80,2.3,46.0,3,2,2
59,4.190,58.0,9.70,0.6,10.30,24.0,210.0,4,3,4
60,3.500,3.9,12.80,6.6,19.40,3.0,14.0,2,1,1


## Question :

Appliquez une ACP aux données de `mammals` et affichez les valeurs propres.

### Preprocessing step

    We have to scale our data before using our PCA

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data.values)

In [6]:
scaled_df = pd.DataFrame(scaled_data)

In [7]:
scaled_df

,0,1,2,3,4,5,6,7,8,9
0,7.237778,5.883379,-0.078244,-0.108003,-1.636219,1.087707,3.578151,0.088109,1.621215,0.270776
1,-0.221768,-0.299686,-0.719848,0.047722,-0.504267,-0.861297,-0.679755,0.088109,-0.891668,0.270776
2,-0.219094,-0.258613,-0.078244,-0.108003,0.446574,-0.318320,-0.552653,-1.277583,-0.891668,-1.128234
3,-0.221858,-0.300661,-0.078244,-0.108003,1.352135,-0.255448,-0.799795,1.453801,-0.263447,0.270776
4,2.632885,4.681532,-2.034353,-0.108003,-1.500385,2.825235,3.429866,0.088109,1.621215,0.970281
...,...,...,...,...,...,...,...,...,...,...
57,-0.220647,-0.293509,-1.158016,-1.120214,-1.160799,-0.689831,0.435915,0.088109,-0.891668,0.270776
58,-0.222773,-0.304129,1.439696,0.514897,1.193662,-0.987039,-0.651510,0.088109,-0.263447,-0.428729
59,-0.218191,-0.243983,0.344275,-1.042352,-0.051486,0.253236,0.506527,0.770955,0.364773,0.970281
60,-0.218965,-0.302612,1.314505,3.629393,2.008668,-0.947030,-0.877468,-0.594737,-0.891668,-1.128234



### Sklearn PCA

In [8]:
from sklearn.decomposition import PCA

In [9]:
sklearn_pca = PCA(n_components=2)

In [10]:
reduced_data = sklearn_pca.fit_transform(scaled_df.values)#ou sklearn_pca.fit_transform(scaled_data)

In [11]:
reduced_data

array([[ 6.50530263,  6.7330641 ],
       [-0.42443283, -0.80647942],
       [-1.56762174,  0.6345905 ],
       [-0.60859835, -1.12707436],
       [ 6.19361561,  4.17407302],
       [ 1.32018225, -0.99632356],
       [-3.4292    ,  1.41344843],
       [ 2.84382529, -0.69307734],
       [-1.96542103,  1.23919915],
       [-0.13357458,  1.67456411],
       [ 0.36715152, -1.59531354],
       [ 3.58409296, -1.3457594 ],
       [-1.04669831,  0.20950559],
       [ 3.17140474, -0.8098289 ],
       [-1.28244543,  0.30247268],
       [ 0.22000675,  0.14454502],
       [-1.28463223,  0.13276622],
       [-0.6515225 , -0.08946067],
       [ 0.02525931,  0.58455006],
       [-3.41813771,  1.6155924 ],
       [ 2.96781394, -0.77358455],
       [ 2.91793753, -2.18090134],
       [-2.05648618, -0.07778988],
       [ 0.22804031,  1.45558916],
       [ 1.04712846,  1.29117978],
       [-1.53982149,  0.74367027],
       [-1.26003136, -0.84114996],
       [ 0.91540347, -1.91026209],
       [ 4.21830947,

In [12]:
print(data.shape)
print(reduced_data.shape)

(62, 10)
(62, 2)


In [13]:

print("Pourcentage de variance expliquée : ")
print(sklearn_pca.explained_variance_ratio_)
print("Composantes principales : ")
print(sklearn_pca.components_)

Pourcentage de variance expliquée : 
[0.48017524 0.22078825]
Composantes principales : 
[[ 0.2507328   0.28338618 -0.32111983 -0.29060816 -0.38538159  0.24828001
   0.37336221  0.25423678  0.38127144  0.33119293]
 [ 0.4417748   0.47478054  0.09769085  0.23679916  0.09113044  0.30384009
   0.26672903 -0.42588243 -0.13611297 -0.37700212]]


Chaque ligne de `pca.components_` représente le vecteur unitaire qui donne la direction d’un axe factoriel. Le vecteur `pca.components_[i,:]` correspond à la valeur propre `pca.explained_variance_ratio_[i]`.

In [14]:
## Question à remetre avant la séance prochaine:

Appliquez une ACP aux données de `mammals` avec n_components>2 et affichez les valeurs propres et comparer les explained_variance_ratio_ .

SyntaxError: invalid syntax (<ipython-input-14-e87952d98355>, line 3)